In [1]:
# Colab notebook: GharKa Guide — RAG chatbot using your knowledge_base
# Cell 1: Install requirements
!pip install --quiet faiss-cpu sentence-transformers tiktoken ollama

In [2]:
# Cell 2: Imports and config
import os
import json
from typing import List, Dict
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import ollama
from IPython.display import display, Markdown

# --- Ensure Ollama is running locally ---
print("All packages are installed")

Ensure Ollama is running locally (e.g., run 'ollama run llama3' in a terminal).


In [3]:
# Cell 3: Knowledge base
knowledge_base = [
    {
      "category": "General Finance",
      "questions": [
        "What is personal finance?",
        "How can I improve my financial health?",
        "What are the basic steps for financial planning?",
        "Why is budgeting important?",
        "How do I set financial goals?",
        "What is the 50/30/20 rule?",
        "How much should I save every month?",
        "What is the difference between saving and investing?",
        "How do I build an emergency fund?",
        "How do I get out of debt quickly?",
        "What is credit score and why is it important?",
        "How do I improve my credit score?"
      ],
      "answers": [
        "Personal finance is the management of your money, including budgeting, saving, investing, and planning for retirement.",
        "To improve financial health, track expenses, create a budget, build savings, reduce debt, and invest wisely.",
        "Financial planning includes goal setting, budgeting, saving, investing, tax planning, and retirement planning.",
        "Budgeting is important because it helps you control spending, save money, and plan for future goals.",
        "Set financial goals by making them SMART (Specific, Measurable, Achievable, Relevant, and Time-bound).",
        "The 50/30/20 rule suggests 50% of income for needs, 30% for wants, and 20% for savings/investments.",
        "A good rule is to save at least 20% of your monthly income, but it depends on your goals.",
        "Saving keeps money safe for short-term needs, while investing grows money over time with some risk.",
        "Build an emergency fund with 3-6 months of living expenses in a safe, liquid account.",
        "To get out of debt quickly, use strategies like the snowball method, avalanche method, or debt consolidation.",
        "A credit score measures your creditworthiness and impacts loan approvals and interest rates.",
        "To improve your credit score, pay bills on time, reduce debt, avoid unnecessary credit inquiries, and maintain old accounts."
      ]
    },
    {
      "category": "Budget Optimizer Tool",
      "questions": [
        "What does the budget optimizer do?",
        "How does the budget optimizer work?",
        "Can I input my income and expenses?",
        "What will the tool suggest if I spend too much?",
        "Does the optimizer follow any financial rule?",
        "What is the recommended split between needs, wants, and savings?",
        "Can I customize my budget goals?",
        "Will the optimizer help me reduce debt?",
        "Can the optimizer show me where I’m overspending?",
        "Does the optimizer track my expenses automatically?"
      ],
      "answers": [
        "The budget optimizer helps you manage income and expenses by suggesting an ideal budget allocation.",
        "It analyzes your income and expenses, then provides recommendations to balance needs, wants, and savings.",
        "Yes, you can input your income and expenses, and the optimizer will generate recommendations.",
        "If you overspend, the tool highlights the issue and suggests reducing discretionary spending.",
        "Yes, it follows the 50/30/20 budgeting rule by default but can be customized.",
        "The recommended split is 50% needs, 30% wants, and 20% savings/investments.",
        "Yes, you can adjust your budget goals based on personal needs.",
        "Yes, by suggesting more savings and reduced discretionary spending, it indirectly helps reduce debt.",
        "Yes, it clearly shows overspending categories and suggests where to cut down.",
        "Currently, expenses need to be entered manually, but the tool is designed to guide spending patterns."
      ]
    },
    {
      "category": "Investment Guide Tool",
      "questions": [
        "What does the investment guide do?",
        "What kind of investments does it recommend?",
        "How does the tool decide investment recommendations?",
        "What is risk tolerance and why is it important?",
        "Can the tool suggest short-term and long-term investments?",
        "Does the tool consider age while recommending?",
        "What are low-risk investment options?",
        "What are medium-risk investment options?",
        "What are high-risk investment options?",
        "Does it recommend stocks or mutual funds?",
        "Can it suggest SIPs (Systematic Investment Plans)?",
        "Will the tool tell me how much to invest?",
        "Does the investment guide guarantee returns?",
        "What if I have no prior investment knowledge?",
        "Does the guide help with retirement planning?"
      ],
      "answers": [
        "The investment guide provides personalized investment suggestions based on income, age, and risk profile.",
        "It recommends investments like fixed deposits, mutual funds, SIPs, stocks, bonds, and retirement funds.",
        "It considers your age, income, financial goals, and risk tolerance to suggest investments.",
        "Risk tolerance is how much risk you can handle without stress. It's key to choosing suitable investments.",
        "Yes, it can suggest both short-term safe investments and long-term wealth-building investments.",
        "Yes, age is factored in—young investors are guided toward higher growth, while older ones toward safety.",
        "Low-risk options include fixed deposits, government bonds, and savings schemes.",
        "Medium-risk options include balanced mutual funds, index funds, and SIPs.",
        "High-risk options include stocks, equity funds, and cryptocurrencies (if suitable).",
        "Yes, it suggests stocks and mutual funds depending on your risk appetite.",
        "Yes, SIPs are recommended for disciplined long-term investing.",
        "The tool can suggest approximate investment proportions based on income.",
        "No, investments always carry risk; the tool suggests but doesn’t guarantee returns.",
        "Yes, the guide explains recommendations in simple terms for beginners.",
        "Yes, it provides retirement-oriented suggestions like pension funds, PPF, and long-term bonds."
      ]
    },
    {
      "category": "User Guidance",
      "questions": [
        "How do I use these tools together?",
        "Which tool should I start with?",
        "What if my income is low?",
        "What if I have a lot of debt?",
        "How can I use the optimizer and guide monthly?",
        "Do I need financial knowledge to use these tools?",
        "Can these tools replace a financial advisor?",
        "Are these tools suitable for students?",
        "Can these tools help with family budgeting?",
        "Can I use this for retirement planning?"
      ],
      "answers": [
        "Start with the Budget Optimizer to manage expenses, then use the Investment Guide to grow savings.",
        "Begin with the Budget Optimizer to ensure you’re saving enough, then move to investments.",
        "If income is low, focus on budgeting, cutting unnecessary expenses, and slowly building savings.",
        "If you have debt, prioritize debt repayment while maintaining a small emergency fund.",
        "Yes, track your income/expenses monthly in the optimizer, then check investments with the guide.",
        "No prior financial knowledge is required—the tools provide simple, guided recommendations.",
        "These tools are meant for guidance, not as a substitute for licensed financial advisors.",
        "Yes, students can use them to manage allowances or part-time income.",
        "Yes, families can track household expenses and plan investments together.",
        "Yes, especially the Investment Guide, which gives retirement-friendly investment suggestions."
      ]
    }
]

# Cell 4: Flatten KB into docs (one doc per Q/A pair)
docs = []
for section in knowledge_base:
    cat = section["category"]
    qs = section.get("questions", [])
    ans = section.get("answers", [])
    # Pair question and answer lists in order where possible
    for i, q in enumerate(qs):
        a = ans[i] if i < len(ans) else ""
        docs.append({
            "category": cat,
            "question": q,
            "answer": a,
            "text": f"Category: {cat}\nQ: {q}\nA: {a}"
        })

print(f"Prepared {len(docs)} knowledge docs.")

# Cell 5: Create embeddings for docs using sentence-transformers
embedder = SentenceTransformer("all-MiniLM-L6-v2")  # small and fast
doc_texts = [d["text"] for d in docs]
embeddings = embedder.encode(doc_texts, show_progress_bar=True, convert_to_numpy=True)

# Build FAISS index
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)  # cosine-like via normalized vectors
# normalize embeddings for cosine similarity
faiss.normalize_L2(embeddings)
index.add(embeddings)
print("FAISS index built with sentence-transformers embeddings.")

# Cell 6: Retrieval helper
def retrieve(query: str, k: int = 4):
    q_emb = embedder.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, k)
    results = []
    for idx, score in zip(I[0], D[0]):
        if idx < 0 or idx >= len(docs):
            continue
        results.append({
            "score": float(score),
            "doc": docs[idx]
        })
    return results

# Cell 7: Chat generation helper using Ollama
def generate_answer(question: str, retrieved: List[Dict], model: str = "llama3", temp: float = 0.1):
    # Compose system + context
    system_prompt = (
        "You are GharKa Guide, a friendly, concise assistant that uses the provided knowledge-base "
        "contents to answer user questions about finance, budget optimizer, investment guidance, and user guidance. "
        "When possible, cite the category of the knowledge used and include the text 'Source: <category>' at the end."
    )
    # Build context from retrieved docs
    context_parts = []
    for r in retrieved:
        d = r["doc"]
        context_parts.append(f"- ({d['category']}) Q: {d['question']}  A: {d['answer']}")
    context_text = "\n".join(context_parts) if context_parts else "No relevant KB content found."

    user_prompt = (
        f"User question:\n{question}\n\n"
        f"Relevant knowledge base excerpts:\n{context_text}\n\n"
        "Using only the knowledge above and general safe reasoning, produce a clear, accurate, and concise answer. "
        "If the KB doesn't contain a direct answer, provide a helpful best-effort answer and mention that it's a suggestion. "
        "End with 'Source: <category>' lines for each KB excerpt used (or 'Source: none' if none)."
    )

    # Call Ollama
    try:
        response = ollama.chat(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            options={"temperature": temp, "num_predict": 500}
        )
        out = response['message']['content'].strip()
    except Exception as e:
        out = f"Error calling Ollama: {e}. Ensure Ollama is running locally with the model loaded."
    return out

# Cell 8: Single-call function to ask question (interactive)
def ask(question: str, k: int = 4, show_retrieval: bool = True):
    retrieved = retrieve(question, k=k)
    if show_retrieval:
        display(Markdown("**Retrieved snippets (score descending):**"))
        for r in retrieved:
            doc = r["doc"]
            display(Markdown(f"- **{doc['category']}** — Q: {doc['question']}  \n  A: {doc['answer']}  \n  (score: {r['score']:.3f})"))
    answer = generate_answer(question, retrieved)
    display(Markdown("### Answer from GharKa Guide"))
    display(Markdown(answer))
    return answer

# Cell 9: Try it out (example)
print("Try questions like: 'How do I build an emergency fund?' or 'What is the 50/30/20 rule?'")
example_q = "How do I build an emergency fund?"
# ask(example_q)

# Cell 10: Simple loop for repeated queries
print("You can call ask('your question here') repeatedly. Example: response = ask('What is personal finance?')")

Prepared 47 knowledge docs.


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

FAISS index built with sentence-transformers embeddings.
Try questions like: 'How do I build an emergency fund?' or 'What is the 50/30/20 rule?'
You can call ask('your question here') repeatedly. Example: response = ask('What is personal finance?')


In [4]:
# Cell 11: Colab Chat Interface
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets

# --- UI setup ---
input_box = widgets.Text(
    value='',
    placeholder='Type your question and press Enter...',
    description='You:',
    layout=widgets.Layout(width='90%')
)
output_area = widgets.Output()

# --- on_submit handler ---
def on_submit(text_widget):
    question = text_widget.value.strip()
    if not question:
        return
    output_area.append_display_data(Markdown(f"**🧑 You:** {question}"))
    text_widget.value = ""  # clear input
    with output_area:
        print("🤔 Thinking...")
        try:
            answer = ask(question, k=4, show_retrieval=False)
        except Exception as e:
            print(f"⚠️ Error: {e}")

# Bind the handler
input_box.on_submit(on_submit)

# --- Display interface ---
display(HTML("<h2>🏠 GharKa Guide Chatbot</h2><p>Ask me about budgeting, finance, or investing!</p>"))
display(output_area)
display(input_box)

C:\Users\rosha\AppData\Local\Temp\ipykernel_25720\2581860345.py:29: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_box.on_submit(on_submit)


Output()

Text(value='', description='You:', layout=Layout(width='90%'), placeholder='Type your question and press Enter…

In [ ]:
# Cell 12: Simplified Chat Loop
def ask_gharka_guide(question):
    """Ask GharKa Guide any question and get an LLM-powered answer."""
    try:
        # Combine all knowledge base entries into one context
        context = "\n\n".join([
            f"Category: {entry['category']}\nQuestions: {', '.join(entry['questions'])}\nAnswers: {', '.join(entry['answers'])}"
            for entry in knowledge_base
        ])

        prompt = f"""
You are 'GharKa Guide', a friendly financial assistant chatbot.
You answer user questions about finance, budgeting, and investments using the knowledge base below.

Knowledge Base:
{context}

User question: {question}
Answer clearly and conversationally as GharKa Guide:
"""

        # Call Ollama
        response = ollama.chat(
            model="llama3",
            messages=[{"role": "user", "content": prompt}],
            options={"temperature": 0.1, "num_predict": 500}
        )
        return response['message']['content'].strip()

    except Exception as e:
        return f"⚠️ Error calling Ollama: {e}. Ensure Ollama is running."

# --- Simple Loop to Chat ---
print("🏠 GharKa Guide Chatbot is ready! Type 'exit' to stop.\n")

while True:
    user_input = input("💬 You: ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("👋 Goodbye! Thanks for chatting with GharKa Guide.")
        break

    answer = ask_gharka_guide(user_input)
    print(f"🤖 GharKa Guide: {answer}\n")

🏠 GharKa Guide Chatbot is ready! Type 'exit' to stop.



💬 You:  What does the budget optimizer do


🤖 GharKa Guide: The Budget Optimizer is a fantastic tool that helps you manage your income and expenses like a pro! It's designed to suggest an ideal budget allocation based on your unique financial situation. By analyzing your income and expenses, it provides personalized recommendations to balance your needs, wants, and savings.

Think of it as having a personal finance coach in your corner, helping you make the most of your hard-earned money. The optimizer takes into account your financial goals, income, and spending habits to give you actionable advice on how to allocate your funds effectively.

Whether you're trying to pay off debt, build an emergency fund, or simply get a better grip on your finances, the Budget Optimizer is here to help. So, what are you waiting for? Give it a try and start optimizing your budget today!



💬 You:  what is the 50/30/20 rule


🤖 GharKa Guide: The 50/30/20 rule! It's a super helpful guideline for allocating your income towards different expenses. Essentially, it suggests that you should divide your monthly income into three categories: needs, wants, and savings/investments.

* 50% of your income goes towards needs (essentials like rent/mortgage, utilities, groceries, transportation, and minimum payments on debts). This is the foundation of your budget!
* 30% for wants (discretionary spending like entertainment, hobbies, travel, and lifestyle upgrades). This is where you can indulge in things that bring you joy!
* 20% for savings/investments (building an emergency fund, retirement planning, or investing in stocks, bonds, or other assets). This is crucial for securing your financial future!

Remember, this rule is just a starting point. You might need to adjust the proportions based on your individual circumstances, such as high-interest debt or specific financial goals. The key is to find a balance that works 